In [1]:
from IPython.display import clear_output, display

import requests
from bs4 import BeautifulSoup
import json

import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np

import json
import time
import mouse
import os
from os.path import exists
import re


In [ ]:
# Основное тело сайта
main_url = 'https://mytclub.ru'

# Страница для получения категорий
sigarets_url = f'{main_url}/sigarety/'

# Парсим html
response = requests.get(sigarets_url)
soup = BeautifulSoup(response.text, 'lxml')

# Получаем нужные элементы в коде
lvl0_elements = soup.find('div', attrs={'id': 'js-sidebar-block'})\
                        .find_all('li', attrs={'class': "nav-item dropdown"})
# Получаем нужные значения
lvl0 = [category.find('a').text for category in lvl0_elements]
lvl0_urls = [category.find('a').attrs['href'] for category in lvl0_elements]
all_data = {(category.find('a').text, category.find('a').attrs['href']): {cat.find('a').text: cat.find('a').attrs['href'] \
                                                                          for cat in category.find_all('li')} \
                for category in lvl0_elements}
print(all_data)

# Формируем переменные-балванки для добавления элементов столбцов
lvl0_list = list()
lvl1_list = list()
lvl1_url_list = list()

# Создаем списки для формирования столбцов таблицы
for k, v in all_data.items():
    lvl0 = k[0]
    if not v:
        lvl1 = k[0]
        lvl1_url = k[1]
        lvl1_list.append(lvl1)
        lvl1_url_list.append(f'{main_url}{lvl1_url}')
        lvl0_list.append(lvl0)
    else:
        for k1, v1 in v.items():
            lvl1 = k1
            lvl1_url = v1
            lvl1_list.append(lvl1)
            lvl1_url_list.append(f'{main_url}{lvl1_url}')
            lvl0_list.append(lvl0)

# Формируем таблицу с категориями
sig_categories = pd.DataFrame({'lvl0': lvl0_list,
                               'lvl1': lvl1_list,
                               'lvl1_url': lvl1_url_list})

# Сохраняем таблицу с категориями
sig_categories.to_excel('./data/mytclub_categories.xlsx', index=False)

In [81]:
# Подгружаем таблицу с полученными категориями
sig_categories = pd.read_excel('./data/mytclub_categories.xlsx')
sig_categories

,lvl0,lvl1,lvl1_url
0,Сигариллы,Сигариллы,https://mytclub.ru/sigarilly/sigarilly/
1,Кальяны,Аксессуары для кальянов,https://mytclub.ru/kalyany/aksessuary-dlya-kal...
2,Кальяны,Табаки для кальяна,https://mytclub.ru/kalyany/tabaki-dlya-kalyana/
3,Кальяны,Уголь для кальяна,https://mytclub.ru/kalyany/ugol-dlya-kalyana/
4,Кальяны,Кальяны,https://mytclub.ru/kalyany/kalyany/
5,Аксессуары для курения,Зажигалки,https://mytclub.ru/aksessuary-dlya-kureniya/za...
6,Аксессуары для курения,Поднос,https://mytclub.ru/aksessuary-dlya-kureniya/po...
7,Аксессуары для курения,Гриндер,https://mytclub.ru/aksessuary-dlya-kureniya/gr...
8,Аксессуары для курения,Ерши для трубок,https://mytclub.ru/aksessuary-dlya-kureniya/ye...
9,Аксессуары для курения,Машинки для самокруток,https://mytclub.ru/aksessuary-dlya-kureniya/ma...


In [102]:
def get_max_page_num_requests(url: str) -> int:
    '''
    Возвращает номер последней страницы в категории
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        max_page = soup.find_all('ul', attrs={'class': "s-paging-list"})[0]\
                        .find_all('li')[-2].text.strip()
    except IndexError:
        max_page = 1
    
    return int(max_page)


def get_url_pagenation(url: str, page_filter: str='page') -> list:
    '''
    Генерирует список страниц для посещения
    '''
    max_page = get_max_page_num_requests(url)
    return [f'{url}?{page_filter}={page_num}' for page_num in range(1, max_page + 1)]


def get_page_data(parsed_soup: BeautifulSoup) -> dict:
    '''
    Получаем данные со страницы
    '''
    item_names = [el.text for el in parsed_soup.find_all('p', attrs={'class': "s-product-header"})]
    
    return {'item_name': item_names}


In [104]:
# File name for saving
file_name = './data/mytclub_data.csv'

if exists(file_name):
    data_table = pd.read_csv(file_name, sep='\t')
    data_table['body_url'] = data_table.apply(lambda x: x['lvl_url'].split('?')[0], axis=1)
    url_body_list = data_table['body_url'].unique().tolist()
    url_list = data_table[data_table['body_url'] == url_body_list[-1]]['lvl_url'].unique().tolist()
else:
    url_body_list = []
    url_list = []

# Цикла для всех категорий
for url in sig_categories.iloc[:, -1].tolist():
    # Проверка уже спарсенных категорий
    if url not in url_body_list[:-1]:
        # Цикл для всех страниц из категории
        for i, paged_url in enumerate(get_url_pagenation(url=url, page_filter='page')):
            
            # Проверка уже спарсенных страниц в катгории
            if paged_url not in url_list:

                # Очистка вывода
                if not i % 5:
                    clear_output()
                print(paged_url)

                # Парсим текущую страницу
                response = requests.get(paged_url)
                soup = BeautifulSoup(response.text, 'lxml')

                # Получаем необходимые данные
                data_dict = get_page_data(soup)

                # Выводим данные
                print(list(zip(*data_dict.values()))[-1], end='\n\n')

                # Добавляем столбец с адресом
                parsed_data = pd.DataFrame(data_dict)
                parsed_data.insert(0, 'lvl_url', paged_url)

                # Сохраняем полученные данные
                parsed_data.to_csv(file_name, 
                                   mode='a' if exists(file_name) else 'w', 
                                   sep='\t', 
                                   index=False,
                                   header = not exists(file_name))
            else:
                continue
    else:
        continue
clear_output()
